In [1]:
import numpy as np, seaborn as sns, pandas as pd
from matplotlib import pyplot as plt
import matplotlib.image as img
import glob

Step 1: Obtain face images I1, I2, ..., IM (training faces)

Step 2: Represent each image as a vector.

In [2]:
path = r'./Train1' # use your path
all_images = glob.glob(path + "/*.jpg")

A = []

for image in all_images:
    arr = img.imread(image).flatten()
    A.append(arr)
    
A = np.asarray(A)
A

array([[130, 128, 127, ..., 129, 129, 129],
       [133, 124, 129, ..., 130, 130, 130],
       [127, 129, 133, ..., 129, 129, 129],
       ...,
       [129, 129, 130, ..., 128, 128, 128],
       [129, 128, 129, ..., 129, 129, 129],
       [129, 129, 129, ..., 129, 129, 129]], dtype=uint8)

Step 3: Compute the average face vector Ψ

In [3]:
mean_vector = np.mean(A, axis=0, dtype = np.float64)
mean_vector

array([129.04931071, 130.16675504, 129.71182397, ..., 129.21818664,
       129.21818664, 129.21818664])

Step 4: Subtract the mean face.

In [4]:
A = np.subtract(A, mean_vector)
A

array([[ 0.95068929, -2.16675504, -2.71182397, ..., -0.21818664,
        -0.21818664, -0.21818664],
       [ 3.95068929, -6.16675504, -0.71182397, ...,  0.78181336,
         0.78181336,  0.78181336],
       [-2.04931071, -1.16675504,  3.28817603, ..., -0.21818664,
        -0.21818664, -0.21818664],
       ...,
       [-0.04931071, -1.16675504,  0.28817603, ..., -1.21818664,
        -1.21818664, -1.21818664],
       [-0.04931071, -2.16675504, -0.71182397, ..., -0.21818664,
        -0.21818664, -0.21818664],
       [-0.04931071, -1.16675504, -0.71182397, ..., -0.21818664,
        -0.21818664, -0.21818664]])

Step 6: Get M best eigenvalues and eigenvectors of A * A^T

In [5]:
A = A.T #redefine each image within the column
eig_vals, eig_vects = np.linalg.eig(np.matmul(A.T, A))

Step 7: Get K best eigenvalues and eigenvectors.
For future people, each row corresponds to an eigenvector and its' eigenvalue at the end.

In [6]:
#this is our K-value
K = 20

test = pd.DataFrame(eig_vects.T)
test2 = pd.Series(eig_vals)
test['Eigenvalues'] = test2

result = test.nlargest(K, ['Eigenvalues'], keep='first')